### Prerequisites
- Python 3
- Python Libraries
    - json, logging, os, time, re, html, matplotlib.pyplot, numpy, pandas, itertools, statistics
- Selenium and Firefox Webdriver
- Jupyter Notebooks

## Running the Code


### Summary

At the end of the notebook run `do_it_all(firstname, lastname)` You must **look at the browser window** that pops up or the program will not run. There are optional parameters for this function for determining the weights of the different parameters. I decided that choosing those was outside of the scope of this project since the choice of those variables really depends on a lot of factors and I just put in some somewhat arbitrary values. A lot more details can be found in the README.md or in the abstract, but this is all you should need to get things running. 

truth finder is sometimes a bit finicky but it should be working now

In [ ]:
import json
import logging
import os.path
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import html
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools
from statistics import mean

In [ ]:
import matplotlib as mp
mp.rcParams['axes.prop_cycle'] = mp.cycler(color=["#FF5733", "#C70039", "#900C3F", "#581845"]) 

In [ ]:
test = False

In [ ]:
def remove_middle(name):
    #just drops everything that isn't the first and last space separated word
    s = name.split(" ")
    if len(s)>2:
        return s[0]+" "+s[-1]
    return name

In [ ]:

def spokeo(driver,name, timeout = 30):
    driver.get("https://www.spokeo.com")
    name_f = '//*[@id="homepage_hero_form"]/div/div[1]/input'
    inputElement = driver.find_element_by_xpath(name_f)
    inputElement.send_keys(name)
    submit = '//*[@id="search"]'
    inputElement = driver.find_element_by_xpath(submit)
    inputElement.click()
    s = time.time()
    while True:
        found = "/html/body/div/div[2]/div/div/div[2]/div[3]/div/div[1]/div[1]/label"
        if (time.time() - s > timeout):
            print("could not be found")
            return None
        try:
            inputElement = driver.find_element_by_xpath(found)
            break
        except:
            sleep(0.2)
    
    return driver  


def spokeo_cont(driver):
    
    #print(list_html)
    d = {}
    q = 2
    base = 0
    button_exists = True
    old_html = driver.page_source
    og_url = driver.current_url.split("?")[0]
    while(button_exists):
        people_list = "/html/body/div/div[2]/div/div/div[2]/div[3]/div/div[2]"
        try:
            peopleElement = driver.find_element_by_xpath(people_list)
            list_html = (peopleElement.get_attribute('innerHTML'))
        except:
            return d
        for i,person in enumerate( peopleElement.find_elements_by_class_name('single-column-list-item.css-kr37en.e468cfn0')):
            p_d = {}
            #print(person.get_attribute('href'))
            #maybe also click on checkout link to get the map and the address

            inner_text = (person.get_attribute('innerHTML'))


            #print(inner_text)

            #name
            name = re.findall("<div class=\"title css-1xq4bjx edso92o0\">[^<]*</div>", inner_text)[0].lower()[40:-6]

            if len(name)>0:

                s = name.split(",")
                if len(s)>1:
                    name = s[0]
                    age = s[1]
                    p_d["name"] = name.replace(".","").replace("<strong>","").replace("</strong>","")
                    p_d["age"] = int(age)
                    #print(name, age)
                elif (len(s)==1):
                    p_d["name"] = s[0]

                else:
                    print("This is the wild wast",s)


            aka = re.findall("Also known as</strong><span>[^<]*<", inner_text)#[0][27:-2]
            
            if len(aka)>0 and aka[0]!="":
                s = set(aka[0][28:-1].lower().replace(".","").replace("<strong>","").replace("</strong>","").split(", "))
                s.add(p_d["name"])
                p_d["aka"] = s
                #print(p_d["aka"])
                
                if len(p_d["aka"])>0:
                    s=set()
                    for nm in p_d["aka"]:
                        s.add(remove_middle(nm))
                    p_d["aka_no_mid"] = s

            current = re.findall("Resides in [^<]*<", inner_text)
            if len(current)>0:
                p_d["current"] = current[0][11:-1].lower().replace(",","").replace("<strong>","").replace("</strong>","")#.split(", ")
                #print(p_d["current"])


            previous = re.findall("Lived In</strong><span>[^<]*</span>",inner_text)
            if len(previous)>0:
                s = set(previous[0][23:-7].lower().replace("<strong>","").replace("</strong>","").split(", "))
                if len(current)>0:
                    s.add( p_d["current"])
                p_d["previous"] = s
                
                #print(p_d["previous"])


            related_to = re.findall("Related To</strong><span>[^<]*</span>", inner_text)
            if len(related_to)>0:
                p_d["related_to"] = set(related_to[0][25:-7].lower().replace(".","").replace("<strong>","").replace("</strong>","").split(", "))
                #print(p_d["related_to"])
                
                if len(p_d["related_to"])>0:
                    s=set()
                    for nm in p_d["related_to"]:
                        s.add(remove_middle(nm))
                    p_d["related_to_no_mid"] = s

            d[base+i] = p_d
        base = base+i+1
        next_button = "/html/body/div/div[2]/div/div/div[2]/div[3]/div/div[3]/a[8]"
        try:
            #c = "/html/body/div/div[2]/div/div/div[2]/div[3]/div/div[3]/a[12]"
            #inputElement = driver.find_element_by_xpath(next_button)
            #inputElement = driver.find_element_by_xpath('//a[contains(@href,"NEXT")]')
            #inputElement.click()
            #print(q)
            if q>10:
                button_exists = False
            else:
                driver.get(og_url+"/"+str(q)+"?loaded=1")
                time.sleep(3)
                
                if not driver.current_url.split("?")[0].endswith(str(q)):
                    button_exists = False
                
                else:
                
                    q+=1
                    new_html = driver.page_source

                    if new_html == old_html:
                        button_exists = False
                    old_html = new_html
                
        except:
            
            button_exists = False
    return d


    """driver.execute_script("window.open('');")
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[1])"""
    
    
    
    
    
    
def anywho(driver,name):
    driver.get("https://infotracer.com/name/results/?page=results&fullname="+name.lower().replace(" ","+")+"&state=all&type=name")
    

    
def anywho_continue(driver):
    
    people_list = "/html/body/div[1]/div[2]/div/div[2]/table/tbody"
    try:
        peopleElement = driver.find_element_by_xpath(people_list)
    #peopleElement = driver.find_element_by_css_selector("tbody")
    except:
        people_list = "/html/body/div[1]/div[2]/div/div[1]/table/tbody"
        try:
            peopleElement = driver.find_element_by_xpath(people_list)
        #peopleElement = driver.find_element_by_css_selector("tbody")
        except:
            return {}
    #list_html = (peopleElement.get_attribute('innerHTML'))
    #print(list_html)
    d = {}
    
    #for i,person in enumerate( peopleElement.find_elements_by_xpath('/html/body/div[1]/div[2]/div/div[2]/table/tbody/tr')):
    for i,person in enumerate( peopleElement.find_elements_by_xpath(people_list+"/tr")):
        p_d = {}
        
        
        
    
        
        html_txt = (html.unescape(person.get_attribute('innerHTML')))
        
        
        name = re.findall('<div class="r2-name"><a href="[^"]*">[^<]*</a></div>', html_txt)
        
        if len(name)>0:
            name = re.findall('>[^<]*</a>', name[0])
            if len(name)>0:
                p_d["name"] = name[0][1:-4].lower().replace("  "," ").replace(".","").replace("<strong>","").replace("</strong>","")
                #print(p_d["name"])
            else:
                print("noname1")
        else:
            print("noname2")
        
        
        
        #print(html_txt)
        
        
        
        try:
            age = (person.get_attribute('data-filter-age'))
            if age!="":
                p_d["age"] = int(age)
        except:
            pass
        
        
        
        try:
            related_to = (person.get_attribute('data-filter-relatives'))
            r = (related_to.lower().replace(".","").replace("<strong>","").replace("</strong>","").split(","))
            if len(r)>0 and r[0]!="":
                r = set(r)
                r.add(p_d["name"])
                p_d["related_to"] = r
                #print(p_d["related_to"])
                if len(p_d["related_to"])>0:
                    s=set()
                    for nm in p_d["related_to"]:
                        s.add(remove_middle(nm))
                    p_d["related_to_no_mid"] = s
        except:
            pass
        
        

        aka = re.findall('<div class="r2-aka"><a href="[^"]*">AKA: [^<]*</a></div>', html_txt)
        if len(aka)>0:
            aka = re.findall('AKA: [^<]*</a>', aka[0])
            #print(aka)
            if len(aka)>0:
                a = set(aka[0][5:-4].lower().replace(".","").replace("<strong>","").replace("</strong>","").split(", "))
                a.add(p_d["name"])
                p_d["aka"] = a
                #print(a)
                
            
                if len(p_d["aka"])>0:
                    s=set()
                    for nm in p_d["aka"]:
                        s.add(remove_middle(nm))
                    p_d["aka_no_mid"] = s
                    
        
        current_prev_rel = re.findall('<div class="r2-list"><a href="[^"]*">(.*?)</a></div>', html_txt)
        if len(current_prev_rel)==2:
            current_prev = current_prev_rel[0]
            
            
            rel = current_prev_rel[1]
            rels = re.findall('<li>(.*?)</li>', rel)
            
        elif len(current_prev_rel)==1:
            #only have one thing in it
            if "strong" in current_prev_rel[0]:
                current_prev = current_prev_rel[0]
                rel = ""
                rels = []
            else:
                current_prev = ""
                rel = current_prev_rel[0]
                rels = re.findall('<li>(.*?)</li>', rel)
                
            
        else:
            current_prev = ""
            rel = ""
            rels = []
            
            
        #stuff
        places = re.findall('<li>(.*?)</li>', current_prev)
        if len(places)>0:
            p_set = set()
            for p in places:
                if "strong" in p:
                    z = p[8:-9].lower().replace(",","")
                    p_d["current"] = z
                    p_set.add(z)
                    #print("current",p_d["current"])
                else:
                    p_set.add(p.lower().replace(",",""))
            if len(p_set)>0 and p_set!=set(""):
                p_d["previous"] = p_set
                #print("previous",p_d["previous"])
            else:
                #print("places",places)
                #means that they only have the current
                pass
            
            
            
        if len(rels)>0 and rels[0]!="": 
            related_to = set()
            for p in (rels):
                related_to.add(p.lower().replace(".","").replace("<strong>","").replace("</strong>",""))
            p_d["related_to"] = related_to
            #print(p_d["related_to"])
            if len(p_d["related_to"])>0:
                s=set()
                for nm in p_d["related_to"]:
                    s.add(remove_middle(nm))
                p_d["related_to_no_mid"] = s
        else:
            pass
            #no relatives
            #print("rels",rels,rel)
        
        
        
        
        d[i] = p_d
        
    return d    
        
    
def addresses(driver,name):
    #blocked
    driver.get("https://www.addresses.com/white-pages/"+name.lower().replace(" ", "+"))
    
def peep(driver,name):
    #blocked
    driver.get("https://www.peoplelookup.com/search/"+name.lower().replace(" ", "-"))
    
def white_pages(driver, name):
    #crappy and doesn't work
    driver.get('https://www.whitepages.com')
    inputElement = driver.find_element_by_xpath('//*[@id="primary_person_search_who"]')
    
    inputElement.send_keys(name)
    
    inputElement = driver.find_element_by_xpath('/html/body/div[5]/div[2]/div[1]/div/div/div/form/button')
    inputElement.click()   
    
    
def truth_finder(driver,first,last):
    sep = "%20"
    url = "https://www.truthfinder.com/results/?firstName="+first.lower().replace(" ",sep)+"&lastName="+last.lower().replace(" ",sep)+"&city=&state=ALL&gender=skip&qLocation=false&qRelatives=false&qAgeRange="
    driver.get(url)
    time.sleep(3)
    #print(driver.page_source)

def truth_cont(driver):
    
    html = driver.page_source.replace("\n","")
    d = {}
    """<div class="age-range-blurb">
<p>
<span class="verified-name-match">Verified name match: <span class="check"></span><strong>Alexandra Nisenoff </strong> </span>
<span class="verified-age-match" style="display: none;">Verified age match:<span class="check"></span><strong>Age <span class="age-range"></span></strong></span>
</p>
</div>"""
    #for i,person  in enumerate(re.findall('<div class="person(.*?)Open Report', html)):
    for i,person  in enumerate(re.findall('<div class="person(.*?)<div class="age-range-blurb">', html)):
        p_d = {}
        #print(i,person)
        
        #'name': 'alexandra marie nisenoff'
        name = re.findall('<h4 class="link-name">(.*?) </h4>', person)
        if len(name)>0:
            name = name[0]
            #print(name)
            if ('<img src="//' in name):
                l =   re.findall('<im(.*?)>', name)
                name = name[len(l[0])+4:]
            else:
                name = name[:-3]
                
            p_d["name"] = name.lower().replace(".","")
            #print("|"+p_d["name"]+"|")
        else:
            print("no_name")
        
        
        #'age': 21,
        try:
            age = re.findall('<span class="display-age">(.*?)</span>', person)
            if len(age)>0:
                p_d["age"] = int(age[0]) 
        except:
            pass
        
        
        #'related_to': {'jeffrey david nisenoff'},
        #'related_to_no_mid': {'jeffrey nisenoff'},
        

        rel = re.findall('<li class="relatives"><ul><li>(.*?)</li> </ul>', person)
        #print(rel)
        if len(rel)>0:
            #print(rel[0])
            rel = rel[0].replace("<span>","").replace("</span>","").lower().replace(",","").split("</li><li>")
            s = set(rel)
            s.add(p_d["name"])
            p_d["related_to"] = s
            #print(s)
            if len(p_d["related_to"])>0:
                s=set()
                for nm in p_d["related_to"]:
                    s.add(remove_middle(nm))
                p_d["related_to_no_mid"] = s
        else:
            pass
        current = re.findall('<div class="address">(.*?) </div>', person)
        #print(current)
        if len(current)>0:
            p_d["current"] = current[0].lower().replace(",","")
        
        
        
        #'previous': {'waikoloa hi'}}}
        prev = re.findall('<li class="location"><ul><li class="include">Locations Include: </li><br><li>(.*?)</li> </ul>', person)
        #print(prev)
        if len(prev)>0:
            prev = prev[0].replace("<span>","").replace("</span>","").lower().replace(",","").split("</li><li>")
            s = set(prev)
            s.add(p_d["current"])
            p_d["previous"] = s
            #print(s)
            
        
        
        

        
    
        d[i] = p_d
    
    #print(html)
    return d
    
    
if test:
    driver = webdriver.Firefox()
    driver.get('https://www.spokeo.com/John-Doe?loaded=1')
    s_dict = spokeo_cont(driver)
    print(s_dict)  

In [ ]:
if test:
    driver3 = webdriver.Firefox()
    truth_finder(driver3,"john","smith")
    t_dict = truth_cont(driver3)
t_dict

In [ ]:
if test:
    driver = webdriver.Firefox()
    driver.get('http://google.com/')
    spokeo(driver,"john smith")
    s_dict = spokeo_cont(driver)
    s_dict

In [ ]:
if test:
    driver2 = webdriver.Firefox()
    driver2.get('http://google.com/')
    anywho(driver2,"john smith")
    a_dict = anywho_continue(driver2)
    a_dict

In [ ]:
def exact_name(match, check):
    #print(match,"|", check)
    #you must give a first and last name (unless the person does not have a last name)
    #will not differentiate between second first name, middle name, and second last name
    match = match.lower().split(" ")
    check = check.lower().split(" ")
    m_l = len(match)
    c_l = len(check)
    
    if m_l == c_l:
        correct = True
        for i,x in enumerate(match):
            #print(x,check[i],x==check[i],correct and (x==check[i]))
            correct = correct and (x==check[i])
        return correct
    elif m_l>c_l:
        return False
    else:
        correct = True
        ofset = 0
        for i,x in enumerate(match):
            or_thing = False
            while i+ofset<c_l:
                or_thing = or_thing or x==check[i+ofset]
                if x==check[i+ofset]:
                    break
                ofset+=1
            
            correct = correct and or_thing
        
        
        return correct and match[0]==check[0] and match[-1]==check[-1]
    
    
if test:   
    print(exact_name("amy swan", "amy swan"),"True")
    print(exact_name("amy swan", "amy e swan"),"True")
    print(exact_name("amy person swan", "amy person swan"),"True")

    print(exact_name("amy person swan", "amy person other swan"),"True")
    print(exact_name("amy person swan", "amy other person swan"),"True")

    print(exact_name("amy person cat swan", "amy person other cat swan"),"True")
    print(exact_name("amy person cat swan", "amy person cat other swan"),"True")


    print(exact_name("amy cat swan", "amy other person swan"),"False")
    print(exact_name("amy cat swan", "amy other person swan"),"False")

    print(exact_name("amy person swan", "amy notperson swan"),"False")
    print(exact_name("amy swan", "ami e swan"),"False")
    print(exact_name("amy", "ami e swan"),"False")
    
    print(exact_name("amy swan", "something amy swan"),"False")

In [ ]:
def comp_counts(dl,dlName,name):
    assert len(dl)==len(dlName)
    

    N = len(dl)
    ind = np.arange(N)
    width = 0.5

    
    exact_names = [sum([(exact_name(name, d[person]["name"])) for person in d]) for d in dl]
    all_match = [len(x) for x in dl]
    not_exact_name = [a - b for a, b in zip(all_match, exact_names)]


    p1 = plt.bar(ind, exact_names, width)
    p2 = plt.bar(ind, not_exact_name, width, bottom=exact_names)

    plt.ylabel('Count')
    plt.title('Comparison of Names Per Record')
    plt.xticks(ind, dlName)
    #plt.yticks(np.arange(0, 81, 10))
    plt.legend((p1[0], p2[0]), ('exact name matches', 'not exact name matches'))

    plt.show()

    
if test:
    comp_counts([s_dict,a_dict,t_dict],["Spokeo", "infotracer","truthfinder"],"john smith")
    
#number of responses with exact name 


In [ ]:
def comp_counts_all(dl,dlName):
    assert len(dl)==len(dlName)
    
    

    #name = "jnadsld"
    N = len(dl)
    ind = np.arange(N)
    width = 0.5
    
    """for d in dl:
        for p in d:
            print(d[p]["name"],exact_name(name, d[p]["name"]))
    """
     
    
    


    have_name = []
    have_current = []
    have_have_lived = []#beyond one
    location_count = []
    have_rel = []
    rel_count = []
    have_aka = []
    aka_count = []
    have_age = []
    
    ages_list = []
    aka_max = []
    rel_max = []
    location_max = []
    
    
    """
        {'age': 41,
          'related_to': ,
          'name': 'amy swan',
          'aka': ,
          'current': ,
          'previous': }
        """
    for  d in dl:
        have_name_tmp = []
        have_current_tmp = []
        have_age_tmp = []
        have_rel_tmp = []
        have_aka_tmp = []
        aka_count_tmp = []
        rel_count_tmp = []
        location_count_tmp = []
        have_have_lived_tmp = []#beyond one
        ag_l = []
        
        aka_max_tmp = 0
        rel_max_tmp = 0
        location_max_tmp = 0
        
        
        
        ld=len(d)
        
        for p in d:
            #True if "name"in d[p] else False
            have_name_tmp.append(True if "name" in d[p] else False)
            have_current_tmp.append(True if "current" in d[p] else False)
            have_age_tmp.append(True if "age" in d[p] else False)
            have_rel_tmp.append(True if "related_to" in d[p] else False)
            have_aka_tmp.append(True if "aka" in d[p] else False)
            have_have_lived_tmp.append(True if (len(d[p].get("previous",[0]))-1)>0 else False)
            
            
            al = len(d[p].get("aka",[0]))-1
            aka_count_tmp.append(al)#if there is a aka list the normal name will be added
            aka_max_tmp = max(aka_max_tmp,al)
            
            rl = len(d[p].get("related_to",[]))
            rel_count_tmp.append(rl)
            rel_max_tmp = max(rel_max_tmp,rl)
            
            ll = len(d[p].get("previous",[0]))-1
            location_count_tmp.append(ll)
            location_max_tmp = max(location_max_tmp, ll)
            
            
            
            if "age" in d[p]:
                ag_l.append(d[p]["age"])  
            
        
        #````````````````````````````
        have_name.append(sum(have_name_tmp)/ld if ld!=0 else 0)
        have_current.append(sum(have_current_tmp)/ld if ld!=0 else 0)
        have_age.append(sum(have_age_tmp)/ld if ld!=0 else 0)
        have_rel.append(sum(have_rel_tmp)/ld if ld!=0 else 0)
        have_aka.append(sum(have_aka_tmp)/ld if ld!=0 else 0)
        aka_count.append(mean(aka_count_tmp) if ld!=0 else 0)
        rel_count.append(mean(rel_count_tmp) if ld!=0 else 0)
        location_count.append(mean(location_count_tmp) if ld!=0 else 0)
        
        have_have_lived.append(sum(have_have_lived_tmp)/ld if ld!=0 else 0)#beyond one
        
        aka_max.append(aka_max_tmp)
        rel_max.append(rel_max_tmp)
        location_max.append(location_max_tmp)
        
        ages_list.append(ag_l)
        
        
        
        #plot have it
    
    group_list_have = ['Have Name','Have Current Location','Have Location beyond Current','Have Relatives','Have Alias', 'Have Age']
    index_l = [(1+i)*2 for i,name in enumerate(group_list_have)]
    
    a=np.array([have_name,have_current,have_have_lived,have_rel,have_aka,have_age])
    
    #print(a)
    df=pd.DataFrame(a, columns=dlName, index=index_l)
    
    
    ax = df.plot(kind='bar')
    ax.set_xticklabels(group_list_have, rotation=90)
    
    plt.ylabel('Percent of records with this data type')
    plt.title('Prevalence of different data types across data brokers')
    plt.show()    
    
    
    #plot counts
    group_list_have = ['Aliases','Relatives','Locations beyond Current']
    index_l = [(1+i)*2 for i,name in enumerate(group_list_have)]
    
    a=np.array([aka_count,rel_count,location_count])
    
    #print(a)
    df=pd.DataFrame(a, columns=dlName, index=index_l)
    
    
    ax = df.plot(kind='bar')
    ax.set_xticklabels(group_list_have, rotation=0)
    
    plt.ylabel('Average Count')
    plt.title('Average count of data of this type for a single record')
    plt.show()
    
    
    
    
    

    
    
    #plot max
    group_list_have = ['Aliases','Relatives','Locations beyond Current']
    index_l = [(1+i)*2 for i,name in enumerate(group_list_have)]
    
    a=np.array([aka_max,rel_max,location_max])
    
    #print(a)
    df=pd.DataFrame(a, columns=dlName, index=index_l)
    
    
    ax = df.plot(kind='bar')
    ax.set_xticklabels(group_list_have, rotation=0)
    
    plt.ylabel('Max Count')
    plt.title('Max count of data of this type for a single record')
    plt.show()
    
    
    
    
    
    
    #plot hist of ages
    for i,nm in enumerate(dlName):
        #print(nm,ages_list[i])
        if ages_list[i]!=[]:
            b = max((max(ages_list[i])-min(ages_list[i])),1)
            plt.hist(ages_list[i],bins=b, alpha=0.5, label=nm)
        
        
        
    plt.legend(loc='upper right')
    plt.ylabel('Count')
    plt.title('Distribution of Result Ages')
    plt.show()
    
    
if test:
    comp_counts_all([s_dict,a_dict,t_dict],["Spokeo", "Infotracer","Truthfinder"])

In [ ]:
def jaccard_slow(Ta,Tb):
    #Ta = tokenize(a)
    #Tb = tokenize(b)
    
    return len(Ta.intersection(Tb))/len(Ta.union(Tb))

memoization = {}
def editfast(s,t):
    if (s,t) in memoization:
        return memoization[(s,t)]
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    rtn = min([1 + editfast(s[1:], t), 1 + editfast(s, t[1:]), (s[ 0 ] != t[ 0 ]) + editfast(s[ 1 :], t[ 1 :])])
    memoization[(s,t)] = rtn
    return rtn
if test:
    editfast("cat","cattttt")

In [ ]:
def name_comp(a1,a2):
    if a1==None or a2==None:
        return 0
    if a1==a2:
        return 1
    s1 = a1.split(" ")
    s2 = a2.split(" ")
    
    scr = 0
    
    frac = 1/max(len(s1),len(s2))
    
    if s1[-1]==s2[-1]:
        scr += frac
    else:
        scr+= max(frac-(editfast(s1[-1],s2[-1])/max(len(s1[-1])*2,len(s2[-1])*2)),0)
    if s1[0]==s2[0] and len(s1)>1:
        scr += frac
    else:
        scr+= max(frac-(editfast(s1[0],s2[0])/max(len(s1[0])*2,len(s2[0])*2)),0)
        
    if len(s1)>2 and len(s2)>2:
        m1 = " ".join(s1[1:-1])
        m2 = " ".join(s1[1:-1])
        
        if m1.startswith(m2) or m2.startswith(m1):
            scr += .1
        else:
            scr+= ((1-(frac*2))-(editfast(m1,m2)/max(len(m1),len(m2))))
    
    return min(1,scr)
if test:
    print(name_comp("alexandra nisenoff","alexandra nisenoff"))
    print(name_comp("alexandra nisenoff","alexandr nisenoff"))
    print(name_comp("alexandra nisenoff","alexandra nise\noff"))
    print(name_comp("alexandra nisenoff","alexandra m nisenoff"))

In [ ]:
def current_comp(a1,a2):
    if a1==None or a2==None:
        return 0
    
    if a1==a2:
        return 1
    s1 = a1.split(" ")
    s2 = a2.split(" ")
    
    if s1[-1]!=s2[-1]:
        return 0
    return max(0.5,1-(editfast(" ".join(s1[:-1])," ".join(s2[:-1]))/max(len(a1),len(a2))))
if test:
    print(current_comp("chicago il","chicago il"))
    print(current_comp("chicago0 il","chicago il"))
    print(current_comp("chic4do0 il","chicago il"))
    print(current_comp("chicago hi","chicago il"))
    print(current_comp("somethinf il","chicago il"))


In [ ]:
def all_jac(a1,a2,b1,b2,string):
    if a1==None or a2==None or b1==None or b2==None:
        return 0
    j = jaccard_slow(a1,a2)
    j2 = jaccard_slow(b1,b2)*.8
    
    
    if j>j2 and j>0:
        pass#print(string,j,a1,a2)#
    elif j2>j and j2>0:
        pass#print(string,j2,b1,b2)#
        
    return max(j, j2)

In [ ]:
def have_lived_comp(a1,a2):
    if a1==None or a2==None :
        return 0
    j = jaccard_slow(a1,a2)
    if j>0:
        pass#print("have lived",j,a1,a2)#
    return j

def relatives_comp(a1,a2,b1,b2):
    return all_jac(a1,a2,b1,b2,"rel")

def aka_comp(a1,a2,b1,b2):
    return all_jac(a1,a2,b1,b2,"aka")
    

In [ ]:
def age_comp(a1,a2):
    #print(a1,a2)
    if a1==None or a2==None:
        return 0
    return max(0,1-(abs(a1-a2)/100))
if test:
    print(age_comp(100,100))
    print(age_comp(100,101))
    print(age_comp(102,101))

In [ ]:
def aka_name_comp(p1aka,p1name,p2aka,p2name):
    if p1name==p2name:
        return 1
    r = 0
    if p2aka!=None and (p1name in p2aka):
        r+=.5
    if p1aka!=None and p2name in p1aka:
        r+=.5
        
    return r

In [ ]:

def table(d1,d2,n_c=1,c_c=1,prev_liv_c=1,rel_c=2,ak_c=1,ag_c=.5):
    d1_key = []
    d2_key = []
    name_sim = []
    current_sim = []
    have_lived_sim = []
    relatives_sim = []
    aka_sim = []
    age_sim = []
    total_sim = []
    
    
    for j,(i1,i2) in enumerate(itertools.product(d1.keys(), d2.keys())):
        p1 = d1[i1]
        p2 = d2[i2]
        
        d1_key.append(i1)
        d2_key.append(i2)
        
        
        
        
        n_sim = name_comp(p1.get("name",None),p2.get("name",None))
        name_sim.append(n_sim)
        
        c_sim = current_comp(p1.get("current",None),p2.get("current",None))
        current_sim.append(c_sim)
        
        
        
        prev_liv_sim = have_lived_comp(p1.get("previous",None),p2.get("previous",None))
        have_lived_sim.append(prev_liv_sim)
        
        
        #kasdlfjlasdjf    related_to_no_mid
        rel_sim = relatives_comp(p1.get("related_to",None),p2.get("related_to",None), 
                                 p1.get("related_to_no_mid",None),p2.get("related_to_no_mid",None))
        relatives_sim.append(rel_sim)
        
        #lksjfl;askdj
        ak_sim = aka_comp(p1.get("aka",None),p2.get("aka",None),
                         p1.get("aka_no_mid",None),p2.get("aka_no_mid",None))
        aka_sim.append(ak_sim)
        
        ag_sim = age_comp(p1.get("age",None),p2.get("age",None))
        age_sim.append(ag_sim)
        
        
        
        
        total_sim.append(sum([(n_sim*n_c),
                              (c_sim*c_c),
                              (prev_liv_sim*prev_liv_c),
                              (rel_sim*rel_c),
                              (ak_sim*ak_c),
                              (ag_sim*ag_c)]))
        
        #print(i1,i2)

    df = pd.DataFrame({"d1_key":d1_key
    ,"d2_key":d2_key
    ,"name_sim":name_sim
    ,"current_sim":current_sim
    ,"have_lived_sim":have_lived_sim
    ,"relatives_sim":relatives_sim
    ,"aka_sim":aka_sim
    ,"age_sim":age_sim
    ,"total_sim":total_sim})
    return df
if test:
    df_all = table(s_dict,a_dict)
    df_all

In [ ]:
def get_max_table(df,cutoff = 0):
    max_df = pd.DataFrame(data=None, columns=df.columns)
    
    d1_u = df["d1_key"].unique().tolist()
    d2_u = df["d2_key"].unique().tolist()
    if len(d1_u)<=len(d2_u):
        l = d1_u
        col = "d1_key"
    else:
        l = d2_u
        col = "d2_key"
        
    for key in l:
        person_df = df[ (df[col]==key)]
        m = max(person_df['total_sim'])
        if m>=cutoff:
            max_row = person_df[person_df['total_sim']==m].iloc[0]
            max_df = max_df.append(max_row)
        #max_df
    return max_df.reset_index().drop(columns="index")
if test:
    max_table = get_max_table(df_all)
    #max_table
    plt.hist(max_table["total_sim"])
    plt.show()

In [ ]:
def do_it_all(first_name, last_name,cutoff = 0,i1 = 0, i2 = 1,n_c=1,c_c=1,prev_liv_c=1,rel_c=1,ak_c=1,ag_c=1):
    full_name = first_name.lower()+" "+last_name.lower()
    driver = webdriver.Firefox()
    spokeo(driver,full_name)
    
    s_dict = spokeo_cont(driver)
    print("Finished with Spokeo")
    time.sleep(5)
   
    anywho(driver,full_name)
    
    a_dict = anywho_continue(driver)
    print("Finished with peoplefinder")
    time.sleep(5)
    
    truth_finder(driver,first_name.lower(),last_name.lower())
    #time.sleep(3)
    t_dict = truth_cont(driver)
    print("Finished with truthfinder")
    time.sleep(5)
    
    driver.quit()
    
    dict_l = [s_dict,a_dict,t_dict]
    #print(t_dict)
    
    comp_counts(dict_l,["Spokeo", "infotracer","truthfinder"],full_name)
    comp_counts_all(dict_l,["Spokeo", "infotracer","truthfinder"])
    
    
    
    df_all = table(dict_l[i1],dict_l[i2],n_c=n_c,c_c=c_c,prev_liv_c=prev_liv_c,rel_c=rel_c,ak_c=ak_c,ag_c=ag_c)
    
    
    
    df_max = get_max_table(df_all,cutoff=cutoff)
    plt.hist(df_max["total_sim"])
    plt.title('Distribution of Max Similarity Score')
    plt.show()
    
    return(df_max,[s_dict,a_dict,t_dict])

In [ ]:
df_max,[s_dict,a_dict,t_dict] = do_it_all("john", "smith")
df_max

In [ ]:
df_max,[s_dict,a_dict,t_dict] = do_it_all("john", "doe")
df_max